### Time to first protest since George Floyd

George Floyd: 26 May 2020
End of dataset: 20 May 2022

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
protests = pd.read_csv('data/daily_all_locs.csv')

In [3]:
weekly_protests = pd.read_csv('data/combined_final_weekly_data.csv')

In [4]:
dist = pd.read_csv('data/dist_matrix_name.csv')
dist = dist.set_index('NAME10')

In [5]:
def temp_spatial_effect(dist, time_col):
    """Mutiplying time since last event by
    distance to that location (using inverse of time
    and distance as effect greater when closer)"""
    
    dist = dist.copy()
    dist_inv = 1/dist
    dist_inv[dist_inv == np.inf] = 1
    
    time_since = [time_col.fillna(0)[x:x+3601] for x in range(0, len(time_col), 3601)]
    time_inv = [[1/a if a!=0 else 1 for a in i] for i in time_since]
    
    temp_sp = []
    for i in range(0, 106):
        temp_sp.append(np.dot(time_inv[i], dist_inv).tolist())

    return [i for a in temp_sp for i in a]

# BLM

In [6]:
weeks = []
for i in range(21):
    weeks.append('2020-' + str(i))

In [7]:
blm_weekly = weekly_protests[~weekly_protests['Year-Week'].isin(weeks)]

### Time since first event

In [8]:
protest_list = blm_weekly['Protest?_x'].to_list()
split_list = [protest_list[x:x+3601] for x in range(0, len(protest_list), 3601)]

In [9]:
time_first = []

for i in range(0, 3601):
    time = []
    weeks = np.nan
    for j in range(0, 106):
        if split_list[j][i] == 0:
            if weeks == np.nan:
                pass
            else:
                weeks = weeks + 1

        elif split_list[j][i] == 1:
            if math.isnan(weeks):
                weeks = 0
            else:
                weeks = weeks + 1

        time.append(weeks)
    time = [np.nan if x == 0 else x for x in time] 
    time_first.append(time)

first_event = []

for i in range(0, 106):
    weekly = []
    for j in time_first:
        weekly.append(j[i])

    first_event.append(weekly)

flat_first_event = [i for a in first_event for i in a] 


blm_weekly['Time since first event'] = flat_first_event 

In [14]:
blm_weekly['Temp*spatial first event (all)'] = temp_spatial_effect(dist, blm_weekly['Time since first event'])
blm_weekly['Temp*spatial last event (all)'] = temp_spatial_effect(dist, blm_weekly['Time since last event_x'])

### Under 50km and same state

In [15]:
dist_50 = dist[dist<50]
blm_weekly['Temp*spatial first event (<50km)'] = temp_spatial_effect(dist_50, blm_weekly['Time since first event'])
blm_weekly['Temp*spatial last event (<50km)'] = temp_spatial_effect(dist_50, blm_weekly['Time since last event_x'])

In [16]:
blm_weekly['State_x']

locs = dist.columns.to_list()
locs = [i.split(',')[1] for i in locs]
locs = [i.strip()[:2] for i in locs]

dist_st = dist.copy()
dist_st.columns = locs
dist_st['State'] = locs

dist_st = dist_st.set_index('State')
dist_st.mask(np.not_equal.outer(dist_st.index, dist_st.columns))

blm_weekly['Temp*spatial first event (same state)'] = temp_spatial_effect(dist_st, blm_weekly['Time since first event'])
blm_weekly['Temp*spatial last event (same state)'] = temp_spatial_effect(dist_st, blm_weekly['Time since last event_x'])

In [17]:
blm_weekly['Temp*spatial first event (state >50km)'] = blm_weekly['Temp*spatial first event (same state)'] - blm_weekly['Temp*spatial first event (<50km)']
blm_weekly['Temp*spatial first event (non-state)'] = blm_weekly['Temp*spatial first event (all)'] - blm_weekly['Temp*spatial first event (same state)']

blm_weekly['Temp*spatial last event (state >50km)'] = blm_weekly['Temp*spatial last event (same state)'] - blm_weekly['Temp*spatial last event (<50km)']
blm_weekly['Temp*spatial last event (non-state)'] = blm_weekly['Temp*spatial last event (all)'] - blm_weekly['Temp*spatial last event (same state)']

Keeping all weeks with 'Protest?' = 0 until first protest occurs (only from week 2020-21 --> 26th May 2020)

In [18]:
m = (blm_weekly['Protest?_x'].eq(1)
       .groupby(blm_weekly['GEOID10_x'])
       .apply(lambda x: ~x.cummax().shift().fillna(False)))

blm_weekly = blm_weekly[m]

In [19]:
blm_weekly['Not immune'] = blm_weekly.groupby('NAME10_x')['Protest?_x'].transform('max')

Adding number of days since George Floyd for those that do have protest

In [20]:
blm_weekly['Time to first BLM protest since GF'] = pd.to_datetime(blm_weekly['EVENT_DATE_x']) - pd.to_datetime('2020-05-26')
blm_weekly['Time to first BLM protest since GF'] = (blm_weekly['Time to first BLM protest since GF']).dt.days

In [21]:
blm_weekly['Weeks since GF'] = (pd.to_datetime(blm_weekly['Year-Week'] + '-0', format='%Y-%U-%w') - pd.to_datetime('2020-20-0', format='%Y-%U-%w')) / np.timedelta64(1, 'W')

In [22]:
states = blm_weekly.State_x.unique().tolist()

In [23]:
for state in states:
    blm_weekly.loc[blm_weekly['State_x'] == state, state] = 1
    blm_weekly[state] = blm_weekly[state].fillna(0)

In [ ]:
blm_weekly.to_csv('data/blm_weekly_first_gf.csv')

### Daily - not currently using this

In [ ]:
blm = protests[protests['Actor.Affiliation'] != 'Alt-Right']
blm['EVENT_DATE'] = pd.to_datetime(blm['EVENT_DATE'])  
blm = blm[(blm['EVENT_DATE'] > '2020-05-25') | (blm['EVENT_DATE'].isna())]
blm = blm.sort_values(by='EVENT_DATE')
blm = blm.drop_duplicates(subset='GEOID10', keep='first')
blm['Time to first protest since GF'] = blm['EVENT_DATE'] - pd.to_datetime('2020-05-26')
blm['Time to first protest since GF'] = blm['Time to first protest since GF'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-05-26'))                                                              
blm['Time to first protest since GF'] = (blm['Time to first protest since GF']).dt.days

# Right wing

In [ ]:
right_weekly = weekly_protests[weekly_protests['Actor.Affiliation_y'] != 'BLM']
right_weekly['EVENT_DATE_y'] = pd.to_datetime(right_weekly['EVENT_DATE_y'])  
right_weekly = right_weekly[(right_weekly['EVENT_DATE_y'] > '2020-05-25') | (right_weekly['EVENT_DATE_y'].isna())]
right_weekly = right_weekly.sort_values(by='EVENT_DATE_y')
right_weekly = right_weekly.drop_duplicates(subset='GEOID10_x', keep='first')

In [ ]:
right_weekly['Time to first right protest since GF'] = right_weekly['EVENT_DATE_y'] - pd.to_datetime('2020-05-26')
right_weekly['Time to first right protest since GF'] = right_weekly['Time to first right protest since GF'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-05-26'))                                                              
right_weekly['Time to first right protest since GF'] = (right_weekly['Time to first right protest since GF']).dt.days

In [ ]:
right_weekly.to_csv('data/right_weekly_first_gf.csv')

# Breonna Taylor

In [ ]:
blm_bt = weekly_protests[weekly_protests['Actor.Affiliation_x'] != 'Alt-Right']
blm_bt['EVENT_DATE_x'] = pd.to_datetime(blm_bt['EVENT_DATE_x'])  
blm_bt = blm_bt[(blm_bt['EVENT_DATE_x'] > '2020-03-12') | (blm_bt['EVENT_DATE_x'].isna())]
blm_bt = blm_bt.sort_values(by='EVENT_DATE_x')
blm_bt = blm_bt.drop_duplicates(subset='GEOID10_x', keep='first')

blm_bt['Time to first BLM protest since BT'] = blm_bt['EVENT_DATE_x'] - pd.to_datetime('2020-03-13')
blm_bt['Time to first BLM protest since BT'] = blm_bt['Time to first BLM protest since BT'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-03-13'))                                                              
blm_bt['Time to first BLM protest since BT'] = (blm_bt['Time to first BLM protest since BT']).dt.days

blm_bt.to_csv('data/blm_weekly_first_bt.csv')

# Ahmaud Arbery 

In [ ]:
blm_aa = weekly_protests[weekly_protests['Actor.Affiliation_x'] != 'Alt-Right']
blm_aa['EVENT_DATE_x'] = pd.to_datetime(blm_aa['EVENT_DATE_x'])  
blm_aa = blm_aa[(blm_aa['EVENT_DATE_x'] > '2020-02-22') | (blm_aa['EVENT_DATE_x'].isna())]
blm_aa = blm_aa.sort_values(by='EVENT_DATE_x')
blm_aa = blm_aa.drop_duplicates(subset='GEOID10_x', keep='first')

blm_aa['Time to first BLM protest since AA'] = blm_aa['EVENT_DATE_x'] - pd.to_datetime('2020-02-23')
blm_aa['Time to first BLM protest since AA'] = blm_aa['Time to first BLM protest since AA'].fillna(pd.to_datetime('2022-05-20') - pd.to_datetime('2020-02-23'))                                                              
blm_aa['Time to first BLM protest since AA'] = (blm_aa['Time to first BLM protest since AA']).dt.days

blm_aa.to_csv('data/blm_weekly_first_aa.csv')